<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
# %pip install catboost
# %pip install sklearn
# %pip install tqdm
# %pip install phik
# %pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# %pip install seaborn
# %pip install pandas-profiling[notebook]

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [12]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.3


In [7]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [13]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [14]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [36]:
VERSION    = 1
DIR_TRAIN  = './input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = './input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [39]:
# %pip uninstall pandas-profiling
# %pip install pandas-profiling[notebook,html]
# %pip install ruamel-yaml
train_my = pd.read_csv('dftrain.csv') # мой датасет для обучения модели


In [15]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [8]:
#использование профилировщика пандас для изучения тестового датасета
from pandas_profiling import ProfileReport
profile = ProfileReport(test, title="Pandas Profiling Report")
profile.to_file("your_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 17.70it/s]


In [12]:
# train[['bodyType','brand','color','fuelType','modelDate','mileage']].head(3)
train.sample(3)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
82303,Седан,MERCEDES,FAFBFB,дизель,2016.0,220 d 2.0d AT (194 л.с.) 4WD,4.0,2020,SEDAN AUTOMATIC 220,AUTOMATIC,...,LEFT,NaN,NaN,ORIGINAL,True,NaN,3902500.0,2020-09-04T11:21:47Z,NaN,E_KLASSE
86421,Внедорожник 5 дв.,MERCEDES,040001,дизель,2001.0,270 2.7d AT (163 л.с.) 4WD,5.0,2003,ALLROAD_5_DOORS AUTOMATIC 270,AUTOMATIC,...,LEFT,NaN,3.0,ORIGINAL,True,"{'year': 2014, 'month': 3}",550000.0,2018-06-14T19:16:59Z,NaN,M_KLASSE
48424,Седан,NISSAN,CACECB,бензин,2001.0,2.0 CVT (140 л.с.),4.0,2003,SEDAN VARIATOR 2.0,VARIATOR,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,285000.0,2020-06-07T10:23:52Z,NaN,PRIMERA


In [10]:
test[['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage']]

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,95000
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,58536
...,...,...,...,...,...,...,...,...,...,...,...,...
34681,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1101369...,коричневый,NaN,Продается BMW 316i в отличном состоянии. Прода...,1.6 LTR,136 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,https://autoru.naydex.net/nwI1K7152/b604fdsYg7...,115000
34682,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1101369...,чёрный,NaN,Продаётся отличный автомобиль.,2.0 LTR,190 N12,NaN,дизель,https://autoru.naydex.net/nwI1K7152/b604fdsYg7...,98000
34683,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1101364...,серый,NaN,Хорошее состояние,2.5 LTR,170 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,360000
34684,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x1/11013625...,коричневый,"{""id"":""6336008"",""name"":""xDrive20d Локальная сб...",Автомобиль в идеальном состоянии . Куплен новы...,2.0 LTR,184 N12,"{""engine-proof"":true,""esp"":true,""start-stop-fu...",дизель,https://autoru.naydex.net/nwI1K7152/b604fdsYg7...,90500


In [16]:
# train[['bodyType','brand','color','fuelType','modelDate','mileage']].head(3)
train[['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']].sample(3)

,bodyType,brand,productionDate,engineDisplacement,mileage
24150,Седан,HONDA,2008,2.4,260000
78556,Внедорожник 5 дв.,VOLVO,2008,2.4d,164042
49533,Внедорожник 5 дв.,NISSAN,2013,2.0,67000


In [20]:
inters=set(train.columns).intersection(test.columns)
inters=list(inters)
inters.sort()
print(inters)
dif=list(set(test.columns).difference(inters))
dif.sort()
print(dif)
train.info()

['bodyType', 'brand', 'color', 'description', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']
['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-nul

In [7]:
test['brand'].value_counts()

BMW           4473
VOLKSWAGEN    4404
NISSAN        4393
MERCEDES      4180
TOYOTA        3913
AUDI          3421
MITSUBISHI    2843
SKODA         2741
VOLVO         1463
HONDA         1150
INFINITI       871
LEXUS          834
Name: brand, dtype: int64

In [48]:
train['brand'].value_counts()

MERCEDES      8012
BMW           7393
VOLKSWAGEN    7139
NISSAN        5772
HYUNDAI       5734
KIA           5630
TOYOTA        4824
FORD          4797
AUDI          4084
MITSUBISHI    3933
CHEVROLET     3633
OPEL          3114
SKODA         2806
RENAULT       2713
MAZDA         2311
VOLVO         2073
PEUGEOT       1641
LAND_ROVER    1623
HONDA         1179
CITROEN       1146
LEXUS         1054
DAEWOO        1041
INFINITI      1040
SUBARU         932
SUZUKI         872
SSANG_YONG     717
PORSCHE        713
CHERY          566
JEEP           483
GREAT_WALL     420
DODGE          418
CHRYSLER       323
CADILLAC       322
JAGUAR         313
MINI           312
GEELY          295
Name: brand, dtype: int64

In [46]:
modelDate=test.modelDate.value_counts()
modelDate.loc[modelDate>2]

2010    2503
2013    2275
2008    2243
2011    2173
2006    2169
2009    2098
2012    2047
2014    1918
2005    1805
2007    1733
2015    1638
2002    1218
2000    1207
2017    1134
2004    1032
2016     899
2003     707
1999     644
1995     564
1998     563
1997     528
2001     519
2018     391
1991     354
1996     354
1994     338
1990     294
1992     166
1986     158
1988     153
2019     142
1987     130
1993     111
1989      85
1984      82
1983      68
1982      68
1985      34
1979      26
1975      16
1980      14
1974      11
1978      10
1938       7
1976       6
1981       6
2020       5
1977       5
1972       4
1958       3
1959       3
1955       3
1937       3
Name: modelDate, dtype: int64

In [47]:
productionDate=test.productionDate.value_counts()
productionDate.loc[productionDate>1]

2012    2798
2008    2606
2011    2589
2013    2488
2014    2134
2007    1964
2010    1826
2015    1770
2017    1667
2016    1615
2018    1574
2006    1269
2019    1081
2009     984
2005     969
2004     902
2003     813
2002     702
2001     658
1998     573
1999     557
2000     516
1997     469
2020     260
1996     253
1992     237
1993     217
1995     215
1991     212
1994     198
1990     132
1989      92
1988      87
1987      58
1985      40
1986      38
1984      26
1983      21
1982      15
1980       8
1981       7
1976       3
1978       3
1979       3
1977       3
1972       3
1959       3
1973       2
1953       2
1965       2
1937       2
1969       2
1975       2
Name: productionDate, dtype: int64

In [9]:
test.head(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [10]:
test.sample(3)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
17702,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/c_klas...,серый,NaN,"В хорошем состоянии, все вапросы по телефону",1.8 LTR,122 N12,NaN,бензин,...,SEDAN AUTOMATIC 1.8,автоматическая,EUROPEAN,3 или более,2 года и 6 месяцев,Дубликат,задний,Левый,Не требует ремонта,Растаможен
33026,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/x_trail/...,серебристый,NaN,Продам отличный авто я второй владелец вписан ...,2.0 LTR,141 N12,"{""engine-proof"":true,""cruise-control"":true,""ti...",бензин,...,ALLROAD_5_DOORS VARIATOR 2.0,вариатор,JAPANESE,2 владельца,1 год и 5 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен
22664,седан,LEXUS,https://auto.ru/cars/used/sale/lexus/ls/109724...,чёрный,NaN,В исключительном состоянии. Все вопросы по тел...,5.0 LTR,394 N12,NaN,гибрид,...,SEDAN VARIATOR 5.0,вариатор,JAPANESE,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

## Data Preprocessing

In [40]:
train_my.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              888 non-null    object 
 1   brand                 888 non-null    object 
 2   color                 888 non-null    object 
 3   complectation_dict    230 non-null    object 
 4   engineDisplacement    888 non-null    object 
 5   enginePower           888 non-null    object 
 6   equipment_dict        770 non-null    object 
 7   fuelType              888 non-null    object 
 8   mileage               888 non-null    float64
 9   productionDate        888 non-null    float64
 10  sell_id               888 non-null    int64  
 11  modelDate             888 non-null    float64
 12  model_info            888 non-null    object 
 13  model_name            888 non-null    object 
 14  name                  888 non-null    object 
 15  numberOfDoors         8

In [17]:
train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [18]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']
df_train = train[columns]
df_test = test[columns]
y = train['price']

In [41]:
df_train_my = train_my[columns]
y = train_my['price']

## Label Encoding

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df_train_my['sample'] = 1 # помечаем где у нас трейн
data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [21]:
# для катбуста не нужно собственноручно переводить в категорию
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes

In [22]:
data

,bodyType,brand,productionDate,engineDisplacement,mileage,sample
0,176,29,2014,11,74000,0
1,176,29,2017,27,60563,0
2,176,29,2014,33,88000,0
3,176,29,2014,27,95000,0
4,176,29,2012,33,58536,0
...,...,...,...,...,...,...
123649,57,30,2006,97,260000,1
123650,57,30,2007,97,300000,1
123651,57,30,2014,72,126000,1
123652,57,30,2013,72,84000,1


In [23]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [25]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [26]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 117.95%


# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [27]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [28]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 18.04%


Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [29]:
# попытки разобраться с признаком parse_unixtime
from datetime import datetime
def parseunix(x):
    ts = int(x)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

df_=test.loc[:,['parsing_unixtime']] 
df_['unixtime']=df_['parsing_unixtime'].apply(parseunix)

df_.sort_values(by='unixtime',ascending=True)

,parsing_unixtime,unixtime
9705,1603116197,2020-10-19
10407,1603118942,2020-10-19
10408,1603118945,2020-10-19
10409,1603118949,2020-10-19
10410,1603118953,2020-10-19
...,...,...
34374,1603682685,2020-10-26
34373,1603682679,2020-10-26
34372,1603682676,2020-10-26
34370,1603682667,2020-10-26


In [30]:
np.log(y_train)

53199    13.060488
75086    14.762874
43113    12.762827
69668    12.785491
70499    11.918391
           ...    
6291     14.508657
55136    12.468437
77178    13.217674
865      12.641097
15861    14.403297
Name: price, Length: 71174, dtype: float64

In [31]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [32]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [33]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.05%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [37]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_notmyds_v{VERSION}.csv', index=False)
sample_submission.head(10)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sell_id  34686 non-null  int64  
 1   price    34686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 542.1 KB


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing